**周日——债市日历给债市日历对应的日期中填相应的地方债发行信息**

从QB上导出发行结果的excel后，直接执行上述代码可以生成对应的每周地方债情况，输出格式为

'2023/11/20 - 龙江一般债 178.8亿元、龙江专项债 146.5亿元',


 '2023/11/21 - 青海一般债 20.9亿元、湖北专项债 416.6亿元',


 '2023/11/23 - 新疆一般债 9.2亿元、新疆专项债 8.6亿元、重庆专项债 31.8亿元、天津专项债 294.7亿元、天津一般债 158.5亿元',

 
 '2023/11/24 - 贵州一般债 752.2亿元'

 可能有的一些问题：1.一些发债的省市在债券简称里的名称和实际不符——如黑龙江就是龙江 2.可能有一些新的省份（深圳就是后来加上去的）

In [2]:
import pandas as pd
from collections import defaultdict

file_path = r"C:\Users\GUO\Desktop\qeubee今日发行结果 2024-01-14.xlsx"

xls = pd.ExcelFile(file_path)

local_bond_sheet_name = [name for name in xls.sheet_names if '地方债' in name]
local_bond_data = pd.read_excel(xls, sheet_name=local_bond_sheet_name[0])

local_bond_data['发行日'] = pd.to_datetime(local_bond_data['发行日'], errors='coerce')

structured_bonds = defaultdict(lambda: {'morning': '', 'afternoon': ''})

for _, row in local_bond_data.iterrows():
    date = row['发行日'].strftime('%Y/%m/%d') if not pd.isnull(row['发行日']) else "N/A"
    time_slot = 'morning' if '上午' in row['招标时间'] else 'afternoon'
    bond_info = (f" {row['债券简称']}  {row['类型']} "
                 f"发行量: {row['发行量(亿)']}亿 ")
    structured_bonds[date][time_slot] += bond_info + '; '

provinces = [
    "北京", "天津", "河北", "山西", "内蒙古", "辽宁", "吉林", "龙江",
    "上海", "江苏", "浙江", "安徽", "福建", "江西", "山东", "河南",
    "湖北", "湖南", "广东", "广西", "海南", "重庆", "四川", "贵州",
    "云南", "西藏", "陕西", "甘肃", "青海", "宁夏", "新疆", "深圳",
    "大连"
]

date_bond_summary = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))  

for _, row in local_bond_data.iterrows():
    issue_date = row['发行日'].strftime('%Y/%m/%d') if not pd.isnull(row['发行日']) else "N/A"
    bond_name = row['债券简称']
    bond_type = row['类型']
    bond_amount = row['发行量(亿)']

    province = next((prov for prov in provinces if prov in bond_name), None)
    if province:
        date_bond_summary[issue_date][province][bond_type] += bond_amount

formatted_date_summary = defaultdict(str)

for date, provinces in date_bond_summary.items():
    summary_items = []
    for province, types in provinces.items():
        for bond_type, total_amount in types.items():
            if bond_type in ["一般债券", "一般债"]:
                formatted_bond_type = "一般债"
            elif bond_type in ["项目专项", "普通专项", "专项债"]:
                formatted_bond_type = "专项债"
            else:
                formatted_bond_type = bond_type  

            formatted_amount = round(total_amount, 1)
            summary_items.append(f"{province}{formatted_bond_type} {formatted_amount}亿元")

    formatted_date_summary[date] = '、'.join(summary_items)

formatted_date_summary_list = [f"{date} - {info}" for date, info in formatted_date_summary.items()]

formatted_date_summary_list

c:\Users\GUO\hello\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['2024/01/17 - 河南专项债 246.8亿元', '2024/01/18 - 大连一般债 11.0亿元']

**周一到周四撰写招投标报告中针对上一个交易日预测情况的评价**


必须是某一种格式化好的输入（如prediction_text所示）才能运行。

In [34]:
import pandas as pd
import re
import datetime

today_date = datetime.datetime.now()
days_in_chinese = ["周一", "周二", "周三", "周四", "周五", "周六", "周日"]
day_of_week_chinese = days_in_chinese[today_date.weekday()]

file_path =  r"C:\Users\GUO\Desktop\qeubee今日发行结果 2024-01-14.xlsx"
sheet_name = '国债政金债'
bond_data = pd.read_excel(file_path, sheet_name=sheet_name)

#按照prediction_text给出的格式提取相应的信息
def extract_bond_info_improved(prediction_text):
    bond_names = re.findall(r"(\d+[YM]附息国债\d+|\d+[YM]贴现国债\d+|\d+[YM]国开\d+|\d+[YM]农发\d+)", prediction_text)
    rate_ranges = re.findall(r"(\d+\.\d+)%-(\d+\.\d+)%", prediction_text)

    if len(bond_names) != len(rate_ranges):
        raise ValueError("The number of bond names and rate ranges do not match.")

    bond_keys = {}
    modified_bond_names = {}
    predictions = {}

    for i, bond_name in enumerate(bond_names):
        modified_bond_name = re.sub(r"\d+[YM]", "", bond_name)
        bond_keys[f"Bond{i+1}"] = bond_name
        modified_bond_names[modified_bond_name] = f"Bond{i+1}"
        predictions[modified_bond_name] = tuple(map(float, rate_ranges[i]))
    return bond_keys, modified_bond_names, predictions

prediction_text = "我们预计 1 月 16日 3Y国开02、5Y国开08、10Y农发20、2Y附息国债27、6M贴现国债03、10Y附息国债26分别成交于2.28%-2.33%、2.39%-2.44%、2.28%-2.33%、2.39%-2.44%、2.28%-2.33%、2.39%-2.44%"
bond_keys_improved, modified_bond_names_improved, predictions_improved = extract_bond_info_improved(prediction_text)

#生成“星期几+XX债+发行期数”这半句话
bond_types_translation = {"国开": "国开债", "农发": "农发债", "附息": "国债", "贴现": "国债"}
bond_types = [re.search(r"(\d+[YM])(附息|贴现|国开|农发)", bond).groups() for bond in bond_keys_improved.values()]

bond_counts = {}
for bond in bond_types:
    bond_type = bond_types_translation.get(bond[1], bond[1])
    bond_counts[bond_type] = bond_counts.get(bond_type, 0) + 1

formatted_bond_counts = [f"{bond_type}{count}期" for bond_type, count in bond_counts.items()]
final_output = f"{day_of_week_chinese}" + ''.join(formatted_bond_counts)

#对招标结果进行评价并细节分析，具体评价的输出格式为XX债中标利率怎么样（包含所有发行的利率债），这个判断的逻辑见memo
def analyze_bond_issuance_with_weighted_rate(bond_data, modified_bond_names, predictions):
    assessment = ''
    detailed_assessment = []

    for modified_key, value in predictions.items():
        predicted_range = value
        original_key = bond_keys_improved[modified_bond_names[modified_key]]
        matching_bond = bond_data[bond_data['债券简称'].str.contains(modified_key)]
        if not matching_bond.empty:
            actual_weighted_rate = matching_bond['加权利率'].values[0]

            if 0 < (predicted_range[0] - actual_weighted_rate) <= 0.01 or 0 < (actual_weighted_rate - predicted_range[1]) <= 0.01:
                detailed_assessment.append(f"{original_key}中标利率基本符合我们预期")
            elif actual_weighted_rate < predicted_range[0]:
                detailed_assessment.append(f"{original_key}中标利率低于我们预期")
            elif actual_weighted_rate > predicted_range[1]:
                detailed_assessment.append(f"{original_key}中标利率高于我们预期")
            else:
                detailed_assessment.append(f"{original_key}中标利率符合我们预期")

    if all("高于" in d for d in detailed_assessment):
        assessment = "较差"
    elif all("低于" in d for d in detailed_assessment):
        assessment = "较好"
    elif all("基本符合" in d for d in detailed_assessment):
        assessment = "基本符合我们预期"
    else:
        assessment = "尚可"

    return assessment, detailed_assessment

analysis_result_weighted = analyze_bond_issuance_with_weighted_rate(bond_data, modified_bond_names_improved, predictions_improved)
assessment_str = analysis_result_weighted[0]
detailed_assessment_str = ','.join(analysis_result_weighted[1])

print(f"{final_output}招标结果{assessment_str}，其中{detailed_assessment_str}")


周二国开债2期农发债1期国债3期招标结果尚可，其中3Y国开02中标利率符合我们预期,5Y国开08中标利率符合我们预期,10Y农发20中标利率高于我们预期,2Y附息国债27中标利率低于我们预期,6M贴现国债03中标利率低于我们预期,10Y附息国债26中标利率高于我们预期


c:\Users\GUO\hello\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**周五到期量模板**理论上需要wind换指标的顺序，但我们其实可以简单写个代码这样就不用再调了，可以一直用招投标的顺序

In [1]:
import pandas as pd

file_path = r"C:\Users\GUO\Desktop\新发行债券.xlsx"
df = pd.read_excel(file_path)

desired_columns = [
    "交易代码", "债券简称", "发行起始日", "缴款日", "计划发行规模(亿)", "发行规模(亿)", "发行期限(年)", "特殊期限", "债券评级", 
    "主体评级", "票面利率(%)", "增发债发行收益率(%)", "指导利率(%)", "指导利率利差(%)", "浮动利率(%)", "特殊条款", "增信方式", 
    "上市日期", "上市地点", "债券代码列表", "募集资金用途", "起息日", "到期日", "利率类型", "付息频率", "评级机构", "发行人简称", 
    "发行人全称", "发行人行业(二级)", "发行人企业性质", "发行人省份", "担保人", "发行时担保人评级", "担保人企业性质", "担保条款", 
    "公告日期", "发行截止日", "主承销商", "副主承销商", "簿记管理人", "发行方式", "承销方式", "发行价格", "招标标的", "招标方式", 
    "投标区间", "招标日", "中标价位", "中标区间", "认购倍数", "发行费率(%)", "承销团成员", "上网发行数量", "上网认购代码", 
    "Wind债券类型(二级)", "次级债或混合资本债", "是否发行失败", "是否城投债", "是否增发", "是否跨市场", "债券全称"
]

for col in desired_columns:
    if col not in df.columns:
        df[col] = None  

df = df[desired_columns]

df.to_excel(r"C:\Users\GUO\Desktop\新发行债券.xlsx")


**周五填EPFR的每周情况描述**

需要把country_flow这个表**数值粘贴**到某个新excel里,这样就能输出EPFR的一长段话

In [ ]:
import pandas as pd

def generate_equity_market_summary_final(data, date):
    def format_value(value):
        return f"净流入{value / 100:.2f}亿美元" if value > 0 else f"净流出{abs(value) / 100:.2f}亿美元"

    summary = f"本周权益市场方面，"
    developed_market_trend = "资金流入" if data['发达国家'][date] > 0 else "资金流出"
    emerging_market_trend = "资金流入" if data['新兴市场'][date] > 0 else "资金流出"
    summary += f"发达国家呈现{developed_market_trend}，新兴市场呈现{emerging_market_trend}。"
    summary += f"本周发达国家股票市场{format_value(data['发达国家'][date])}，新兴市场股票市场{format_value(data['新兴市场'][date])}。"

    custom_order = ["美国", "发达欧洲", "日本", "新兴亚洲", "中国", "中国香港", "新兴欧洲", "金砖四国", "印度", "巴西"]
    for category in custom_order:
        if category in data:
            summary += f"{category}股票市场{format_value(data[category][date])}，"

    return summary.strip('，')

file_path = r"C:\Users\GUO\Desktop\新建 Microsoft Excel 工作表.xlsx"

df = pd.read_excel(file_path, sheet_name='Sheet1')

converted_data = {
    row['股票市场资金流向（mn USD）'].strip(): {
        df.columns[i].strftime('%Y/%m/%d'): row[df.columns[i]] for i in range(1, len(df.columns))
    } for index, row in df.iterrows()
}

date_for_summary = "2024/01/03"  # 根据需要更改日期
market_summary = generate_equity_market_summary_final(converted_data, date_for_summary)
print(market_summary)


本周权益市场方面，发达国家呈现资金流入，新兴市场呈现资金流入。本周发达国家股票市场净流入38.02亿美元，新兴市场股票市场净流入38.10亿美元。美国股票市场净流入38.50亿美元，发达欧洲股票市场净流出2.43亿美元，日本股票市场净流入2.29亿美元，新兴亚洲股票市场净流入40.37亿美元，中国股票市场净流入31.59亿美元，中国香港股票市场净流出0.85亿美元，新兴欧洲股票市场净流出0.02亿美元，金砖四国股票市场净流入35.08亿美元，印度股票市场净流入4.16亿美元，巴西股票市场净流出0.64亿美元


In [ ]:
import pandas as pd

def generate_bond_market_summary_final(data, date):
    def format_value(value):
        return f"净流入{value / 100:.2f}亿美元" if value > 0 else f"净流出{abs(value) / 100:.2f}亿美元"

    summary = f"本周债券市场方面，"
    developed_market_trend = "资金流入" if data['发达国家'][date] > 0 else "资金流出"
    emerging_market_trend = "资金流入" if data['新兴市场'][date] > 0 else "资金流出"
    summary += f"发达国家呈现{developed_market_trend}，新兴市场呈现{emerging_market_trend}。"
    summary += f"本周发达国家债券市场{format_value(data['发达国家'][date])}，新兴市场债券市场{format_value(data['新兴市场'][date])}。"

    custom_order = ["美国", "日本", "发达欧洲", "德国","新兴亚洲", "中国", "新兴欧洲", "金砖四国"]
    for category in custom_order:
        if category in data:
            summary += f"{category}债市{format_value(data[category][date])}，"
    return summary.strip('，')

file_path = r"C:\Users\GUO\Desktop\新建 Microsoft Excel 工作表.xlsx"

df = pd.read_excel(file_path, sheet_name='Sheet2')

converted_data = {
    row['债券市场资金流向（mn USD）'].strip(): {
        df.columns[i].strftime('%Y/%m/%d'): row[df.columns[i]] for i in range(1, len(df.columns))
    } for index, row in df.iterrows()
}

date_for_summary = "2024/01/03"  # 根据需要更改日期
market_summary = generate_bond_market_summary_final(converted_data, date_for_summary)
print(market_summary)

本周债券市场方面，发达国家呈现资金流入，新兴市场呈现资金流入。本周发达国家债券市场净流入99.23亿美元，新兴市场债券市场净流入5.86亿美元。美国债市净流入78.25亿美元，日本债市净流入1.30亿美元，发达欧洲债市净流入22.16亿美元，德国债市净流入3.16亿美元，新兴亚洲债市净流入7.09亿美元，中国债市净流入1.40亿美元，新兴欧洲债市净流出0.19亿美元，金砖四国债市净流入0.65亿美元
